### chatcompletition, Prompt Template, LLM api call

In [1]:
import minsearch
import json
import os

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)  

Index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3, 'section' : 0.4}

    results = Index.search(
        query=query,
        filter_dict={'course':'mlops-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering. DO NOT USE OTHER CONTENT OTHER THAN GIVEN CONTEXT!
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [6]:
from openai import OpenAI

In [7]:
def llm_call(prompt):
    client =  OpenAI(
        base_url = "http://localhost:11434/v1",
        api_key = "ollama"
)
    response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt,}],
    model="phi3",
)
    return response.choices[0].message.content

In [8]:

def rag(query):
    
    search_results = search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

In [9]:
rag("How to use Kafka?")

'To evaluate your capstone project: Firstly and primarily, three students who have also submitted their projects will randomly be assigned to you as reviewer. They are mandated by our course policies not only assist with peer review but to provide feedback on the draft itself before final submission of grades for it. Secondarily, based upon the guidelines defined (still TBA), we ask every participant to grade a submitted project from three other students which encourages constructive criticism and comprehensive evaluation among peers themselves beyond random assignment reviewer task; please take this seriously as non-attendance might result in failure for certification. Finally, the scores you receive through both peer review means will be statistically averaged using median method to give final grade of your project at course end; no further evaluations are expected but participation is greatly valued and considered part of educational experience too!\n'

In [ ]:
print(_)